<a href="https://colab.research.google.com/github/ENGRBertoli/webapp00_SITE/blob/main/primeiro_site_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import matplotlib.pyplot as plt
import pickle
import os

# Caminho do arquivo para salvar os dados
DATA_FILE = "balancas_data.pkl"

# Função para carregar os dados
def carregar_dados():
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "rb") as file:
            return pickle.load(file)
    return []

# Função para salvar os dados
def salvar_dados(dados):
    with open(DATA_FILE, "wb") as file:
        pickle.dump(dados, file)

# Lista para armazenar as balanças (carrega do arquivo)
balancas = carregar_dados()

# Função para obter a cor do status
def obter_cor_status(status):
    cores = {
        "Em produção": "orange",
        "Indisponível": "red",
        "Disponível para venda": "green",
        "Vendido": "blue"
    }
    return cores.get(status, "black")

# Função para exibir gráfico de barras verticais
def Grafico_Barra_Status_Vertical(Rotulos, Quantidades, Largura=10, Altura=7, Titulo_Grafico='Gráfico de Status'):
    fig, ax = plt.subplots(figsize=(Largura, Altura))
    colors = [obter_cor_status(r) for r in Rotulos]
    bars = ax.bar(Rotulos, Quantidades, color=colors)
    ax.set_xlabel('Status')
    ax.set_ylabel('Quantidade')
    ax.set_title(Titulo_Grafico)

    for bar, value in zip(bars, Quantidades):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value}', ha='center', va='bottom')

    st.pyplot(fig)

# Função principal do app
def main():
    st.set_page_config(
        page_title="Painel de Gestão de Balanças",
        page_icon="⚖️",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    st.title("Painel de Gestão de Fabricação de Balanças")

    # Menu lateral
    menu = st.sidebar.radio("Escolha uma opção:", ["Adicionar Balança", "Listar Balanças", "Gráficos"])

    if menu == "Adicionar Balança":
        st.subheader("Adicionar uma nova balança")
        modelo = st.text_input("Modelo da balança:").strip()
        quantidade = st.number_input("Quantidade total (deixe em branco para 'Indisponível'):", min_value=0, step=1, value=0)

        if st.button("Adicionar"):
            # Verificar se o modelo já existe
            balanca_existente = next((b for b in balancas if b["modelo"].lower() == modelo.lower()), None)

            if balanca_existente:
                # Atualizar quantidade total e ajustar o status "Em produção"
                balanca_existente["quantidade_total"] += quantidade
                if quantidade > 0:
                    balanca_existente["status_quantidades"]["Em produção"] += quantidade
                st.success(f"A quantidade foi atualizada para o modelo '{modelo}'. Nova quantidade total: {balanca_existente['quantidade_total']}.")
            else:
                # Adicionar novo modelo
                nova_balanca = {
                    "modelo": modelo,
                    "quantidade_total": quantidade,
                    "status_quantidades": {
                        "Em produção": quantidade if quantidade > 0 else 0,
                        "Indisponível": 0 if quantidade > 0 else 1,
                        "Disponível para venda": 0,
                        "Vendido": 0
                    }
                }
                balancas.append(nova_balanca)
                if quantidade > 0:
                    st.success(f"Balança '{modelo}' adicionada com quantidade total de {quantidade}.")
                else:
                    st.warning(f"Balança '{modelo}' adicionada como 'Indisponível'.")

            salvar_dados(balancas)  # Salvar alterações no arquivo

    elif menu == "Listar Balanças":
        st.subheader("Balanças cadastradas")
        if not balancas:
            st.warning("Nenhuma balança cadastrada.")
        else:
            for idx, balanca in enumerate(balancas):
                st.markdown(f"### Modelo: {balanca['modelo']}")
                st.write(f"**Quantidade Total:** {balanca['quantidade_total']}")

                # Ajustar quantidades por status
                for status, quantidade in balanca["status_quantidades"].items():
                    nova_quantidade = st.number_input(
                        f"Quantidade em '{status}' para {balanca['modelo']}:",
                        min_value=0,
                        max_value=balanca['quantidade_total'],
                        value=quantidade,
                        step=1,
                        key=f"{balanca['modelo']}_{status}"
                    )
                    balanca["status_quantidades"][status] = nova_quantidade

                # Verifica se a soma das quantidades por status é válida
                soma_quantidades = sum(balanca["status_quantidades"].values())
                if soma_quantidades > balanca["quantidade_total"]:
                    st.error(f"As quantidades excedem o total de {balanca['quantidade_total']}. Ajuste os valores.")
                elif soma_quantidades < balanca["quantidade_total"]:
                    st.warning(f"As quantidades somadas são menores que o total. Total esperado: {balanca['quantidade_total']}.")
                else:
                    salvar_dados(balancas)  # Salva alterações no arquivo
                    st.success(f"As quantidades para '{balanca['modelo']}' foram atualizadas.")

    elif menu == "Gráficos":
        st.subheader("Gráficos das balanças")
        if not balancas:
            st.warning("Nenhuma balança cadastrada para exibir nos gráficos.")
        else:
            for balanca in balancas:
                st.markdown(f"### Gráfico de Status para {balanca['modelo']}")
                Grafico_Barra_Status_Vertical(
                    list(balanca["status_quantidades"].keys()),
                    list(balanca["status_quantidades"].values()),
                    Largura=10,
                    Altura=7,
                    Titulo_Grafico=f"Status do Modelo {balanca['modelo']}"
                )

if __name__ == "__main__":
    main()




